## Subqueries & Temporary Tables

- Both **subqueries** and **table expressions** are methods for being able to write a query that creates a table, and then write a query that interacts with this newly created table. 


- Sometimes the question you are trying to answer doesn't have an answer when working directly with existing tables in database. However, if we were able to create new tables from the existing tables, we know we could query these new tables to answer our question. 

   - The original query goes in the FROM statement.
   
   - An * is used in the SELECT statement to pull all of the data from the original query.
   
   - You MUST use an alias for the table you nest within the outer query.


### Subquery Formatting Sample Code

```SQL
SELECT channel,
       AVG(event_count) avg_per_channel
FROM (SELECT DATE_TRUNC('day', we.occurred_at) event_date,
      channel,COUNT(channel) as event_count
      FROM web_events we
      GROUP BY 1,2
      ORDER BY 3 DESC) events
GROUP BY 1
ORDER BY 2 DESC
```

###  Using subquery in logical statement

  - For WHERE, HAVING, or even SELECT - the value could be nested within a CASE statement.

  - You should not include an alias when you write a subquery in a conditional statement. This is because the subquery is treated as an individual value (or set of values in the IN case) rather than as a table.



### Common Table Expression
 
 - Break your query into separate components so that your logic is easily readable.
 
 - **The WITH statement**** Although they serve the exact same purpose as subqueries, they are more common in practice, as they tend to be cleaner for a future reader to follow the logic.

```SQL
WITH table1 AS (
          SELECT *
          FROM web_events),

     table2 AS (
          SELECT *
          FROM accounts)


SELECT *
FROM table1
JOIN table2
ON table1.account_id = table2.id;
``` 
 
 
```SQL
WITH events AS 
     (SELECT DATE_TRUNC('day', we.occurred_at) event_date,
      channel,COUNT(channel) as event_count
      FROM web_events we
      GROUP BY 1,2
      ORDER BY 3 DESC) 

SELECT channel,
       AVG(event_count) avg_per_channel
FROM events
GROUP BY 1
ORDER BY 2 DESC
```
  

### PRACTICE -  Subquery Mania 
 
 - **Q1 Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales**
 
 
 - **Subquery** 
 
```SQL
SELECT sales_rep
FROM
    (SELECT r.name region,
       sr.name sales_rep,
       MAX(total_amt_usd) AS total
    FROM region r
    JOIN sales_reps sr
    ON r.id=sr.region_id
    JOIN accounts a
    ON a.sales_rep_id=sr.id
    JOIN orders o
    ON o.id=a.id
    GROUP BY 1,2
    ORDER BY 1) t1

JOIN (SELECT r.name region, 
       MAX(total_amt_usd) AS largest_total
FROM region r
JOIN sales_reps sr
ON r.id=sr.region_id
JOIN accounts a
ON a.sales_rep_id=sr.id
JOIN orders o
ON o.id=a.id
GROUP BY 1
ORDER BY 1) t2
ON t1.total = t2.largest_total

```

- **Common Table** 

```SQL

WITH table1 AS (SELECT r.name region,
                       sr.name sales_rep,
                       MAX(total_amt_usd) AS total
                FROM region r
                JOIN sales_reps sr
                ON r.id=sr.region_id
                JOIN accounts a
                ON a.sales_rep_id=sr.id
                JOIN orders o
                ON o.id=a.id
                GROUP BY 1,2
                ORDER BY 1),

     table2 AS (SELECT r.name region,
                       MAX(total_amt_usd) AS largest_total
                FROM region r
                JOIN sales_reps sr
                ON r.id=sr.region_id
                JOIN accounts a
                ON a.sales_rep_id=sr.id
                JOIN orders o
                ON o.id=a.id
                GROUP BY 1
                ORDER BY 1)
                
SELECT *
FROM table1
JOIN table2
ON table1.total = table2.largest_total
```

 **- Q2 For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?**

 - **Subquery** 
 
```SQL 
SELECT r.name region_name, COUNT(*)
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY 1
HAVING r.name =( SELECT region_name 
                FROM ( SELECT r.name region_name,
                       SUM(o.total_amt_usd) max_sum_of_sales_per_region
                       FROM sales_reps s
                       JOIN accounts a
                       ON a.sales_rep_id = s.id
                       JOIN orders o
                       ON o.account_id = a.id
                       JOIN region r
                       ON r.id = s.region_id
                       GROUP BY 1
                       ORDER BY 2 DESC
                       LIMIT 1) table1 )
```


- **Common Table** 

```SQL

WITH table1 AS (SELECT r.name region_name,
                       SUM(o.total_amt_usd) max_sum_of_sales_per_region
                FROM sales_reps s
                JOIN accounts a
                ON a.sales_rep_id = s.id
                JOIN orders o
                ON o.account_id = a.id
                JOIN region r
                ON r.id = s.region_id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 1),
      table2 AS (SELECT region_name
                 FROM table1
                 )

SELECT r.name region_name, COUNT(*)
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY 1
HAVING r.name=(SELECT * FROM table2)
```



**- Q3 How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?**


 - **Subquery** 
 
```SQL
SELECT COUNT(*) 
FROM (SELECT acc.name account_name,
      SUM(total) total_purchase
      FROM accounts acc
      JOIN orders o
      ON acc.id=o.account_id
      GROUP BY acc,name
      HAVING SUM(total) > (SELECT (total_purchase)
                    FROM (SELECT acc.name account_name,
                          SUM(standard_qty) total_standard_qty_purchased,
                          SUM(total) total_purchase
                          FROM accounts acc
                          JOIN orders o
                          ON acc.id=o.account_id
                          GROUP BY 1
                          ORDER BY 2 DESC
                          LIMIT 1
                          ) t2 ) 
                    ) t3
```


- **Common Table** 

```SQL

WITH table1 AS (SELECT acc.name account_name,
                SUM(standard_qty) total_standard_qty_purchased,
                SUM(total) total_purchase
                FROM accounts acc
                JOIN orders o
                ON acc.id=o.account_id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 1),
     table2 AS  (SELECT total_purchase
                 FROM table1 ),
     table3 AS (SELECT acc.name account_name,
               SUM(total) total_purchase
               FROM accounts acc
               JOIN orders o
               ON acc.id=o.account_id
               GROUP BY acc,name
               HAVING SUM(total) > (SELECT * FROM table2))
               
 SELECT COUNT(*)
 FROM table3
                                    
```


**Q4 -  For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?**

- **Subquery** 

```SQL

SELECT we.channel as channel_name,acc.name as account_name, count(*)
FROM web_events we
JOIN accounts acc
ON acc.id=we.account_id
GROUP BY 1,2
HAVING acc.name= (SELECT biggest_spender 
                 FROM(
                    SELECT acc.name biggest_spender,
                           SUM(total_amt_usd) as lifetime_spending 
                    FROM accounts acc
                    JOIN orders o
                    ON acc.id=o.account_id
                    JOIN web_events we
                    ON acc.id=we.account_id
                    GROUP BY 1
                    ORDER BY 2 DESC 
                    LIMIT 1
                    ) t2) 
```

- **Common Table** 


```SQL

WITH table1 AS (SELECT acc.name biggest_spender,
                           SUM(total_amt_usd) as lifetime_spending 
                FROM accounts acc
                JOIN orders o
                ON acc.id=o.account_id
                JOIN web_events we
                ON acc.id=we.account_id
                GROUP BY 1
                ORDER BY 2 DESC 
                LIMIT 1
                ),
      table2 AS (SELECT we.channel as channel_name,acc.name as account_name, count(*)
                FROM web_events we
                JOIN accounts acc
                ON acc.id=we.account_id
                GROUP BY 1,2
                HAVING acc.name = (SELECT biggest_spender FROM table1 )
                )
SELECT *
FROM table2

```


**- Q5 What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts**

- **Subquery**

```SQL

SELECT AVG(total_spending_top10) as avg_top10
FROM ( 
SELECT a.name, SUM(total_amt_usd) total_spending_top10
FROM accounts a
JOIN orders o
ON a.id = o.account_id 
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
    ) t2
```

- **Common Table** 

```SQL 
WITH table1 AS (SELECT a.name,
                       SUM(total_amt_usd) total_spending_top10
                       FROM accounts a
                JOIN orders o
                ON a.id = o.account_id 
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 10)

SELECT AVG(total_spending_top10) as avg_top10
FROM table1

```


**- Q6 What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.**


- **Subquery**


```SQL

SELECT AVG(spending_above_avg) FROM

( SELECT a.name, AVG(total_amt_usd) spending_above_avg
FROM accounts a
JOIN orders o
ON a.id = o.account_id 
GROUP BY 1
HAVING AVG(total_amt_usd) > (SELECT AVG(total_amt_usd) avg_total
                                   FROM accounts a
                                   JOIN orders o
                                   ON a.id = o.account_id 
                                   ) ) T2
    
```

- **Common Table** 


```SQL

WITH table1 AS (SELECT AVG(total_amt_usd) avg_total
                FROM accounts a
                JOIN orders o
                ON a.id = o.account_id 
                ),      
    table2 AS ( SELECT a.name, AVG(total_amt_usd) spending_above_avg
                FROM accounts a
                JOIN orders o
                ON a.id = o.account_id 
                GROUP BY 1
                HAVING AVG(total_amt_usd) > (SELECT avg_total FROM table1 )
             )
SELECT AVG(spending_above_avg)
FROM table2

```